# **Importations**


In [ ]:
!pip install rpy2==3.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 201 kB 5.2 MB/s 
  Created wheel for rpy2: filename=rpy2-3.5.1-cp38-cp38-linux_x86_64.whl size=310200 sha256=651ccc90be814c21e3529dbdc441fbe6d867e04fbbe78bc1c7868e4fb4c0cc4b
  Stored in directory: /root/.cache/pip/wheels/6b/40/7d/f63e87fd83e8b99ee837c8e3489081c4b3489134bc520235ed
Successfully built rpy2
  Attempting uninstall: rpy2
    Found existing installation: rpy2 3.5.5
    Uninstalling rpy2-3.5.5:
      Successfully uninstalled rpy2-3.5.5


In [ ]:
import pandas as pd
import numpy as np
import csv 
import matplotlib.pyplot as plt
import spacy
import math

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Statapp_TextDigit/Corp_HouseOfCommons_V2.rds'

Mounted at /content/drive


# **Transformation du fichier R en pandas**

In [ ]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
readRDS = robjects.r['readRDS']

In [ ]:
df = readRDS(path)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
len(df[0])

1956223

In [ ]:
df = df.head(5000)

Je prends les n premières lignes pour avoir assez de RAM pour faire la conversion, mais vu que les speeches sont rangés chronologiquement, c'est pas représentatif. 

In [ ]:
df = pandas2ri.rpy2py_dataframe(df)

In [ ]:
df.head(10)

,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country
1,1988-11-22,Queen's Speech,1,CHAIR,None,NaN,1,74.0,I have to acquaint the House that this House h...,UK-HouseOfCommons,GBR
2,1988-11-22,First Day [Debate On The Address],2,CHAIR,None,NaN,1,55.0,It may be for the convenience of the House if ...,UK-HouseOfCommons,GBR
3,1988-11-22,First Day [Debate On The Address],3,Giles Shaw,Con,1567.0,0,2511.0,"I beg to move,. That an humble Address be pres...",UK-HouseOfCommons,GBR
4,1988-11-22,First Day [Debate On The Address],4,John Maples,Con,1567.0,0,1470.0,I am delighted to second the motion. When I ha...,UK-HouseOfCommons,GBR
5,1988-11-22,First Day [Debate On The Address],5,Neil Kinnock,Lab,1516.0,0,2768.0,I am sure that I speak for the majority of hon...,UK-HouseOfCommons,GBR
6,1988-11-22,First Day [Debate On The Address],6,David Harris,Con,1567.0,0,1.0,Appeal.,UK-HouseOfCommons,GBR
7,1988-11-22,First Day [Debate On The Address],7,Neil Kinnock,Lab,1516.0,0,850.0,"The hon. Gentleman says, "" Appeal."" Of course ...",UK-HouseOfCommons,GBR
8,1988-11-22,First Day [Debate On The Address],8,Margaret Thatcher,Con,1567.0,0,189.0,"First, I join the Leader of the Opposition in ...",UK-HouseOfCommons,GBR
9,1988-11-22,First Day [Debate On The Address],9,CHAIR,None,NaN,1,23.0,Order. I did not wish to interrupt the Leader ...,UK-HouseOfCommons,GBR
10,1988-11-22,First Day [Debate On The Address],10,Margaret Thatcher,Con,1567.0,0,37.0,Let me deal quickly with some of the comments ...,UK-HouseOfCommons,GBR


On a donc un dataframe avec 1 956 223 speeches de parlementaires anglais. 

---

On comme information pour chaque speech : 
- la date 
- le sujet 
- le nom du parlementaire
- son parti politique 
- le texte du speech 


*On* réduit le dataframe pour l'instant, pour pouvoir construire le programme plus rapidemment. On pose :

```
df_r = df.iloc[np.random.permutation(df.index)].reset_index(drop=True)
df_r = df_r.head(1000)
```
Donc on mélange d'abord le dataframe, avant de prendre les 1000 premières lignes. 


In [ ]:
df_r = df.sample(frac=1).reset_index(drop=True)
df_r = df_r.head(1000)

In [ ]:
df_r

,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country
0,1988-12-07,Regional Assistance [Oral Answers To Questions...,34,Tony Newton,Con,1567.0,0,68.0,I have no specific additional measures in mind...,UK-HouseOfCommons,GBR
1,1988-12-07,Ec (Trade Statistics) [Oral Answers To Questio...,132,Alan Clark,Con,1567.0,0,71.0,My hon. Friend is entirely right. One problem ...,UK-HouseOfCommons,GBR
2,1988-12-01,Fisheries,312,Donald Thompson,Con,1567.0,0,6.0,I cannot give an absolute assurance.,UK-HouseOfCommons,GBR
3,1988-11-22,First Day [Debate On The Address],11,CHAIR,None,NaN,1,1.0,Order.,UK-HouseOfCommons,GBR
4,1988-11-25,Foreign Affairs And Defence [Orders Of The Day...,35,Gerald Kaufman,Lab,1516.0,0,161.0,"That was lovely, but one of the real jobs of t...",UK-HouseOfCommons,GBR
...,...,...,...,...,...,...,...,...,...,...,...
995,1988-11-30,Angola [Oral Answers To Questions > Foreign An...,65,Lynda Chalker,Con,1567.0,0,51.0,Of course I believe that all parties must come...,UK-HouseOfCommons,GBR
996,1988-11-30,Scottish Transport Group,172,Malcolm Rifkind,Con,1567.0,0,137.0,"As the hon. Gentleman knows, with regard to fe...",UK-HouseOfCommons,GBR
997,1988-11-22,Physiological Measurement Technicians [Debate ...,170,Michael Shersby,Con,1567.0,0,720.0,"My hon. Friend is nodding. If the council has,...",UK-HouseOfCommons,GBR
998,1988-12-07,Water Bill,256,Jack Cunningham,Lab,1516.0,0,8.0,I did not mention the Select Committee's report.,UK-HouseOfCommons,GBR


# **Traitement initial du texte parlementaire**

On va tout d'abord créer une fonction qui va appliquer le même traitement aux speeches qui a été appliqué aux textes des journaux : 
- tokenization 
- nettoyage de la ponctuation et des nombres 
- stemming des mots 
- nettoyage des stopwords à l'aide de la même liste que pour les journaux
- remise des mêmes stopwords significatifs que pour les journaux 

In [ ]:
from spacy.lang.en import English
sp = spacy.load('en_core_web_sm')
import string
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='english')

On utilise le SnowballStemmer.

On importe la liste des stopwords :

In [ ]:
df_stopwords = pd.read_csv("/content/drive/MyDrive/Statapp_TextDigit/Stage_2022/code/news-stopwords-master/sw1k.csv",
                names=['word', 'frequency', 'presence', 'doc_size_sum', 'type'],
                encoding='latin-1').drop(index=0)

In [ ]:
'''
file = open("/content/drive/MyDrive/Statapp_TextDigit/Stage_2022/code/news-stopwords-master/sw1k.csv", "r")
df_stopwords = pd.read_csv(file,
                names=['word', 'frequency', 'presence', 'doc_size_sum', 'type'],
                encoding='latin-1')
df_stopwords = df_stopwords.drop(index=0)
file.close()
'''

'\nfile = open("/content/drive/MyDrive/Statapp_TextDigit/Stage_2022/code/news-stopwords-master/sw1k.csv", "r")\ndf_stopwords = pd.read_csv(file,\n                names=[\'word\', \'frequency\', \'presence\', \'doc_size_sum\', \'type\'],\n                encoding=\'latin-1\')\ndf_stopwords = df_stopwords.drop(index=0)\nfile.close()\n'

On a donc un dataframe des stopwords. On en tire une liste simple des stopwords, où l'on remet cependant des stopwords jugés significatifs, comme expliqué pour le traitement des journaux :

In [ ]:
stopwords = []
putback = ['prime', 'officials', 'security', 'news', 'working', 'games', 'jobs', 'campaign', 'services',
'civil', 'economic', 'information', 'political', 'election', 'court', 'office', 'vote', 'trump', 'control', 'job', 'price',
'donald trump', 'chinese', 'problems', 'concerns', 'minister', 'nation', 'policy', 'data', 'indian', 'congress',
'president', 'network', 'american', 'accused', 'government', 'money', 'investigation',
'facebook', 'success', 'prices', 'twitter', 'book', 'politics',
'justice', 'claims', 'russia', 'law', 'technology', 'content', 'union', 'european', 'workers']

for i in df_stopwords.index :
  if df_stopwords["word"][i] not in putback :
    stopwords.append(stemmer.stem(df_stopwords["word"][i]))
    stopwords.append('I') # pk tu rajoutes des I à ta liste ?

On utilise la librairie SpaCy comme pour le traitement des journaux, et on enlève les nombres et la ponctuation avec la méthode translate. 

In [ ]:
def clean(text):
  text = str(text).lower()
  text = text.translate(str.maketrans('', '', string.punctuation))
  remove_digits = str.maketrans('', '', string.digits)
  text = text.translate(remove_digits) 
  doc = sp(text)
  l = []
  for word in doc :
    word = str(word)
    l.append(stemmer.stem(word))
    if l[-1] == ' ' or l[-1] in stopwords: 
      l.pop()
  return l


In [ ]:
essai = "I'm going to the No.4 shop, shoppi77ng is so 9 like fun computed"
clean(essai)

On peut ensuite appliquer ce nettoyage à nos speeches, et aux sujets de ceux-ci, directement dans le dataframe. 

In [ ]:
df_r['text'] = df_r['text'].apply(clean)
df_r['agenda'] = df_r['agenda'].apply(clean)

# amélioration possible : on enlève le .apply et on vectorise le traitement en modifiant la fonction clean et en l'appliquant direct à la colonne 

In [ ]:
'''
def clean_column(column):
  column = column.str.lower()
  column = column.str.translate(str.maketrans('', '', string.punctuation))
  column.str.translate(str.maketrans('', '', string.digits))
  # j'ai pas encore fini 
  return column

df_r['text'] = clean_column(df_r['text'])
df_r['agenda'] = clean_column(df_r['agenda'])
'''

"\ndef clean_column(column):\n  column = column.str.lower()\n  column = column.str.translate(str.maketrans('', '', string.punctuation))\n  column.str.translate(str.maketrans('', '', string.digits))\n  # j'ai pas encore fini \n  return column\n\ndf_r['text'] = clean_column(df_r['text'])\ndf_r['agenda'] = clean_column(df_r['agenda'])\n"

In [ ]:
df_r

,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country
0,1988-12-07,"[assist, oral, answer]",34,Tony Newton,Con,1567.0,0,68.0,"[specif, addit, measur, sort, hon, gentleman, ...",UK-HouseOfCommons,GBR
1,1988-12-07,"[ec, statist, oral, answer]",132,Alan Clark,Con,1567.0,0,71.0,"[hon, custom, english, sometim, delud, export,...",UK-HouseOfCommons,GBR
2,1988-12-01,[fisheri],312,Donald Thompson,Con,1567.0,0,6.0,"[absolut, assur]",UK-HouseOfCommons,GBR
3,1988-11-22,[debat],11,CHAIR,None,NaN,1,1.0,[],UK-HouseOfCommons,GBR
4,1988-11-25,"[affair, defenc, debat]",35,Gerald Kaufman,Lab,1516.0,0,161.0,"[job, shall, fend, damag, prime, minist, expla...",UK-HouseOfCommons,GBR
...,...,...,...,...,...,...,...,...,...,...,...
995,1988-11-30,"[angola, oral, answer, commonwealth, affair, c...",65,Lynda Chalker,Con,1567.0,0,51.0,"[african, negoti, therefor, sensibl, peac, req...",UK-HouseOfCommons,GBR
996,1988-11-30,"[scottish, transport]",172,Malcolm Rifkind,Con,1567.0,0,137.0,"[hon, gentleman, regard, ferri, gourockdunoon,...",UK-HouseOfCommons,GBR
997,1988-11-22,"[physiolog, measur, technician, debat]",170,Michael Shersby,Con,1567.0,0,720.0,"[hon, nod, kept, profil, investig, recommend, ...",UK-HouseOfCommons,GBR
998,1988-12-07,[],256,Jack Cunningham,Lab,1516.0,0,8.0,"[mention, select]",UK-HouseOfCommons,GBR


Maintenant, chaque speech est transformé en une liste de mots significatifs. 

On voit par exemple que 24 speeches de l'échantillon ont "data" répertorié comme sujet. 

# **Construction du test**

On va tout d'abord essayer de filtrer les speeches qui sont dans le "thème" des nouvelles technologies, big tech, etc. Pour ça, on se créé une liste de mots clés à partir des mots clés utilisés pour les journaux, et on ne garde que les speeches qui sont concernés par ces sujets. On commence par faire une liste assez grossière. 

In [ ]:
technology=['technology','innovat','computer','high tech|high-tech','science','engineering']
consumer_protection=['privacy','data leak','leak','fake news',' safety','decept','defective','hack']
firms=['google','alphabet','apple','facebook','meta','amazon','microsoft']
products=['chrome','incognito','youtube','nexus','pixel','google drive','gmail','glass','street view','buzz','fitbit',
 'maps', 'doodle','play','translate','search', 'google news','nest hub','xl','nest','chromecast','stadia','hub',
 'marshmallow','lollipop','cloud','waymo','earth','engine',

'apple pay','apple watch','iphone','ipad','ipod','iwatch','macbook','macbook pro', 'macbook air','mac',
 'imac','airpods','ios','siri','icloud','apple tv','apple music','app store', 'safari','x','app','apple store',
 'xr', 'xs', 'se','iphones','itunes','ibook','plus','pro','max','mini','os','airtag','airtags','arcade','homepod',
'keynote','ipados','id','foxconn','facetime','beat','stalk',

'messenger','instagram', 'whatsapp','page','feed','oculus',

'prime', 'kindle',
'publishing','amazon prime','amazon drive','amazon video','amazon business','amazon web service',
'amazon cloud', 'alexa','echo dot','echo','dot', 'delivery', 'amazon uk','unlimited', 'episode','foods','grocery', 
'grand tour','grand','tour','viking', 'vikings','argo','argos','macmillan', 'dvd','clarkson','lord','ring','hair','skin','vacuum',
'pre','beer','drake','spark','kart','dog','twitch','cat','xo','matthew','stafford','ratchet','clank',
'swagway','album','mouse','showbiz','beauty','guardian','batman','arkham','gc','hair','skin','shirt',
'lovefilm','mirzapur','cast','audio','drama','movie','jack ryan','actor','character','lucifer','outlander',
'premier','super mario','sky','channel','voyage',


'windows','window','xp','surface','xbox','studio','microsoft office', 'office','word','cortana', 'surface pro','teams',  'playstation',
'microsoft edge', 'edge', 'gear','outlook','halo','skype','kinect','internet explorer','explorer','ie','bing','xcloud','hololens',
'forza','ori','scarlett','scorpio','wordperfect','valhalla','onedrive','games gold','lumia','azure',
'assassin creed','assassin','creed','minecraft','yammer','warcraft','tay']



ceos=['sundar','pichai','eric','schmidt','steve jobs','tim cook','mark zuckerberg','andy jassy','jeff','bezos','satya', 'nadella','bill gates',
      'gates','steve job','steve','tim', 'cook','zuckerberg','ceo','tim cook ','steve ballmer','ballmer','elop',
      'schiller','fadell','phil spencer','spencer','mcspirit','sandberg','paul','allen','larry hryb','hryb']
      

types=['tablet','mobile', 'laptop', 'pc', 'computer', 'desktop','smartphone', 'smartwatch', 'search engine', 'software','hardware',
               'machine', 'browser','ebook', 'book',  'reader',  'console', 'headphone', 'earbud','bud','store','music',
              'gaming', 'operating','streaming','title','chatbot']



competitor=['samsung', 'galaxy',  'twitter','tiktok', 'switch','sony', 'asos', 'activision blizzard', 'activision','blizzard',
            'nintendo','snes', 'netflix','android','yahoo','nokia','huawei','motorola','htc','blackberry','oppo','oneplus','rim','symbian','bbc','morrison','spotify'] 


configue=['device','feature','battery','screen','sound','gb','g','k','mm','chip','processor','design','display','touch','ram',
          'inch','keyboard','camera','handset','speaker','button','touchscreen','storage', 'data']


celebrity=['dubost','neymar','amanda','beyonce','blur','richard','hammond','ranj','jeremy clarkson', 'jeremy','momoa',
           'jared','aniston','smith','kim','tony','tom','sophie','oasis','trio','sharon','betty','raoul','moat','lauren','andrew',
           'samuel gibbs','samuel','gibbs','van','gaal']

topics = celebrity + configue + competitor + types + ceos + products + firms + consumer_protection + technology

On stem les topics : 

In [ ]:
stem_topics = []
for k in range(len(topics)):
  stem_topics.append(stemmer.stem(topics[k]))

In [ ]:
def techno_filter(list):
  for k in range(len(list)):
    if list[k] in stem_topics :
      return True
  return False

In [ ]:
df_r['agenda']= df_r['agenda'].apply(techno_filter)

In [ ]:
df_r

,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country
0,1988-12-07,False,34,Tony Newton,Con,1567.0,0,68.0,"[specif, addit, measur, sort, hon, gentleman, ...",UK-HouseOfCommons,GBR
1,1988-12-07,False,132,Alan Clark,Con,1567.0,0,71.0,"[hon, custom, english, sometim, delud, export,...",UK-HouseOfCommons,GBR
2,1988-12-01,False,312,Donald Thompson,Con,1567.0,0,6.0,"[absolut, assur]",UK-HouseOfCommons,GBR
3,1988-11-22,False,11,CHAIR,None,NaN,1,1.0,[],UK-HouseOfCommons,GBR
4,1988-11-25,False,35,Gerald Kaufman,Lab,1516.0,0,161.0,"[job, shall, fend, damag, prime, minist, expla...",UK-HouseOfCommons,GBR
...,...,...,...,...,...,...,...,...,...,...,...
995,1988-11-30,False,65,Lynda Chalker,Con,1567.0,0,51.0,"[african, negoti, therefor, sensibl, peac, req...",UK-HouseOfCommons,GBR
996,1988-11-30,False,172,Malcolm Rifkind,Con,1567.0,0,137.0,"[hon, gentleman, regard, ferri, gourockdunoon,...",UK-HouseOfCommons,GBR
997,1988-11-22,False,170,Michael Shersby,Con,1567.0,0,720.0,"[hon, nod, kept, profil, investig, recommend, ...",UK-HouseOfCommons,GBR
998,1988-12-07,False,256,Jack Cunningham,Lab,1516.0,0,8.0,"[mention, select]",UK-HouseOfCommons,GBR


In [ ]:
S = 0
for i in df_r.index:
  if df_r['agenda'][i]:
    S+=1
print(S)

52


On voit que sur les 5000 premiers speeches, seuls 62 concernent les topics définis. C'est peu, mais plutôt logique vu que ces speeches ont été faits en 1988 environ. 

On ne garde que ceux-là :

In [ ]:
for i in df_r.index :
  if df_r['agenda'][i] != True :
    df_r = df_r.drop(i)

In [ ]:
df_r

,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country
32,1988-12-08,True,243,Alan Williams,Lab,1516.0,0,128.0,"[speaker, realis, minist, persist, abus, prime...",UK-HouseOfCommons,GBR
34,1988-12-06,True,129,Margaret Thatcher,Con,1567.0,0,97.0,"[yts, guarante, yts, appli, octob, figur, unfi...",UK-HouseOfCommons,GBR
63,1988-11-30,True,198,Stuart Randall,Lab,1516.0,0,155.0,"[speaker, yesterday, prime, minist, abus, proc...",UK-HouseOfCommons,GBR
68,1988-12-13,True,124,John Reid,Lab,1516.0,0,91.0,"[prime, minist, congratul, awar, nurs, lanarks...",UK-HouseOfCommons,GBR
71,1988-12-06,True,108,Neil Kinnock,Lab,1516.0,0,67.0,"[subject, affect, prime, minist, figur, govern...",UK-HouseOfCommons,GBR
114,1988-11-23,True,228,Tony Benn,Lab,1516.0,0,1.0,[answer],UK-HouseOfCommons,GBR
133,1988-12-13,True,114,Margaret Thatcher,Con,1567.0,0,132.0,"[hon, gentleman, agre, repudi, utter, assert, ...",UK-HouseOfCommons,GBR
191,1988-11-23,True,218,Jonathan Aitken,Con,1567.0,0,6.0,"[hon, gentleman]",UK-HouseOfCommons,GBR
213,1988-11-23,True,221,CHAIR,None,NaN,1,58.0,"[hon, freedom, speech, hon, gentlemani, awar, ...",UK-HouseOfCommons,GBR
231,1988-11-30,True,190,CHAIR,None,NaN,1,12.0,"[answer, attorneygener]",UK-HouseOfCommons,GBR


On va repérer les 100 mots (puis 500 quand on aura les speeches en entier) qui sont le plus utilisés par chaque parti, et noter leur fréquence d'apparition. 

In [ ]:
def words_of_party(df, text):
  l = []
  for i in df.index: 
    if df['party'][i] == text :
      l = l + df['text'][i]
  return l

In [ ]:
words_of_Con = words_of_party(df_r, 'Con')
words_of_Lab = words_of_party(df_r, 'Lab')
words_of_LibDem = words_of_party(df_r, 'LibDem')
words_of_SDLP = words_of_party(df_r, 'SDLP')

In [ ]:
def words_frequencies(list):
  words = []
  freqs = []
  for word in list: 
    if word not in words : 
      words.append(word)
      freqs.append(1)
    else :
      k = words.index(word)
      freqs[k] += 1
  d = {'words': words, 'freqs': freqs}
  words_freqs =  pd.DataFrame(data=d)
  return words_freqs

In [ ]:
words_frequencies(words_of_Con)

,words,freqs
0,yts,7
1,guarante,2
2,appli,2
3,octob,1
4,figur,2
...,...,...
362,relev,1
363,document,1
364,dublin,1
365,suffici,1


On a tous les mots utilisés par le parti Conservateur sur les sujets définis, ainsi que leurs fréquences d'apparition. On va retenir les fréquences avec lesquelles chaques mots sont utilisés, par chaque parti, afin de déterminer le chi2 de chaque mot. On se restreint pour l'instant aux partis Con et Lab, fortement majoritaires. 

In [ ]:
 df_freqs = pd.merge(words_frequencies(words_of_Con), words_frequencies(words_of_Lab), on="words", how='outer')

In [ ]:
df_freqs

,words,freqs_x,freqs_y
0,yts,7.0,NaN
1,guarante,2.0,NaN
2,appli,2.0,NaN
3,octob,1.0,NaN
4,figur,2.0,1.0
...,...,...,...
449,spite,NaN,1.0
450,painand,NaN,1.0
451,lifebi,NaN,1.0
452,money,NaN,1.0


In [ ]:
c = 0
for i in range(len(df_freqs['words'])):
  if df_freqs['freqs_x'][i] > 0 and df_freqs['freqs_y'][i] >0 :
    c += 1

In [ ]:
c

57

Il y a donc c mots utilisés par les deux partis. 

In [ ]:
df_freqs

,words,freqs_x,freqs_y
0,yts,7.0,NaN
1,guarante,2.0,NaN
2,appli,2.0,NaN
3,octob,1.0,NaN
4,figur,2.0,1.0
...,...,...,...
449,spite,NaN,1.0
450,painand,NaN,1.0
451,lifebi,NaN,1.0
452,money,NaN,1.0


In [ ]:
len(df_freqs['words'])

454

In [ ]:
l_mots = []
l_chi2 = []
for i in df_freqs.index :
  if math.isnan(df_freqs['freqs_x'][i]) == False :
    fp_Con = float(df_freqs['freqs_x'][i])
  else:
     fp_Con = 0
  if math.isnan(df_freqs['freqs_y'][i]) == False :
    fp_Lab = float(df_freqs['freqs_y'][i])
  else :
    fp_Lab = 0
  not_fp_Con = 0
  not_fp_Lab = 0
  for k in df_freqs.index : 
    if k == i :
      l_chi2 = l_chi2
    else : 
      if math.isnan(df_freqs['freqs_x'][k]) == False :
        not_fp_Con += df_freqs['freqs_x'][k]
      if math.isnan(df_freqs['freqs_y'][k]) == False :
        not_fp_Lab += df_freqs['freqs_y'][k]
  chi2 = ((fp_Con * not_fp_Con - fp_Lab * not_fp_Lab)**2) / ((fp_Con+fp_Lab) * (not_fp_Lab+fp_Lab) * (not_fp_Con+fp_Con) * (not_fp_Con+not_fp_Lab))
  l_mots.append(df_freqs['words'][i])
  l_chi2.append(chi2)



In [ ]:
df_t = pd.DataFrame({'words' : l_mots , 'chi2' : l_chi2})

In [ ]:
df_f = pd.merge(df_freqs, df_t, on = 'words')

In [ ]:
df_f

,words,freqs_x,freqs_y,chi2
0,yts,7.0,NaN,0.025697
1,guarante,2.0,NaN,0.007408
2,appli,2.0,NaN,0.007408
3,octob,1.0,NaN,0.003711
4,figur,2.0,1.0,0.003577
...,...,...,...,...
449,spite,NaN,1.0,0.000331
450,painand,NaN,1.0,0.000331
451,lifebi,NaN,1.0,0.000331
452,money,NaN,1.0,0.000331


On a donc ici chaque mot utilisé par les parti sur le sujet délimité, avec le chi2 correspondant. On va se limiter aux 500 premiers mots :

In [ ]:
df_f = df_f.sort_values(by=['chi2'], ascending=False)

In [ ]:
df_f = df_t.head(500)

In [ ]:
df_f

,words,chi2
0,yts,0.025697
1,guarante,0.007408
2,appli,0.007408
3,octob,0.003711
4,figur,0.003577
...,...,...
449,spite,0.000331
450,painand,0.000331
451,lifebi,0.000331
452,money,0.000331


# **Regréssions Linéaires**

On va implémenter les deux régressions linaires nécessaires en utilisant ```sklearn```: 

In [ ]:
from sklearn import linear_model
import statsmodels.api as sm

On va considérer pour l'instant qu'il n'y a que deux personnes dans le parlement : les Con et les Lab. On note leur idéologie : 1 si ils sont Con, et O si ils sont Lab. On va retourner à la fréquence de chque mot dans le speech de chaque parti. 

In [ ]:
df_Con = pd.merge(words_frequencies(words_of_Con), df_f, on = 'words', how = 'inner')
df_Con = df_Con.assign(freq_in_speech_Con = lambda x: x.freqs / (df_Con['freqs'].sum()))
df_Con

,words,freqs,chi2,freq_in_speech_Con
0,yts,7,0.025697,0.010130
1,guarante,2,0.007408,0.002894
2,appli,2,0.007408,0.002894
3,octob,1,0.003711,0.001447
4,figur,2,0.003577,0.002894
...,...,...,...,...
362,relev,1,0.003711,0.001447
363,document,1,0.003711,0.001447
364,dublin,1,0.003711,0.001447
365,suffici,1,0.000204,0.001447


In [ ]:
df_Lab = pd.merge(words_frequencies(words_of_Lab), df_f, on = 'words', how = 'inner')
df_Lab = df_Lab.assign(freq_in_speech_Lab = lambda x: x.freqs / (df_Lab['freqs'].sum()))
df_Lab

,words,freqs,chi2,freq_in_speech_Lab
0,speaker,4,0.000023,0.019324
1,realis,2,0.000204,0.009662
2,minist,15,0.000007,0.072464
3,persist,2,0.000204,0.009662
4,abus,5,0.001597,0.024155
...,...,...,...,...
139,painand,1,0.000331,0.004831
140,lifebi,1,0.000331,0.004831
141,money,1,0.000331,0.004831
142,vital,1,0.000331,0.004831


In [ ]:
df_reg = pd.merge(df_Con, df_Lab, on = 'words', how = 'outer')
del (df_reg['chi2_x'], df_reg['chi2_y'], df_reg['freqs_x'], df_reg['freqs_y'])
df_reg = df_reg.fillna(0)
df_reg = df_reg.transpose()
new_header = df_reg.iloc[0] 
df_reg = df_reg[1:] 
df_reg.columns = new_header 
df_reg = df_reg.assign(ideology = [1,0])
df_reg

words,yts,guarante,appli,octob,figur,unfil,capabl,entitl,incom,await,...,councillor,eddi,scott,calderdal,spite,painand,lifebi,money,vital,ideology
freq_in_speech_Con,0.01013,0.002894,0.002894,0.001447,0.002894,0.001447,0.001447,0.005789,0.004342,0.001447,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
freq_in_speech_Lab,0.0,0.0,0.0,0.0,0.004831,0.0,0.0,0.0,0.0,0.0,...,0.009662,0.004831,0.009662,0.004831,0.004831,0.004831,0.004831,0.004831,0.004831,0


On a ici la fréquence des 500 mots les plus significatifs dans le speech respectif de chaque parti. 

In [ ]:
# Tentative avec sm

x = df_reg['refer']
y = df_reg['ideology']

x = sm.add_constant(x)

result = sm.OLS(y,x).fit()

print(result.summary())

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


ValueError: ignored

In [ ]:
# Tentative avec sklearn

X = df_reg['govern'].values.reshape(-1, 1)
y = df_reg['ideology'].values.reshape(-1, 1)
lm = linear_model.LinearRegression()
model = lm.fit(X,y)
predictions = lm.predict(X)
print(predictions)

NameError: ignored